In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 26748095
paper_name = 'simoneau_wurtele_2016' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [5]:
original_data = pd.read_excel('raw_data/nar-02322-x-2015-File011.xlsx', sheet_name='Sheet1', skiprows=1)

In [6]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 4679 x 9


In [7]:
original_data.head()

,strain,Gene,"z-score 20 mM NAM generation 5, #1","z-score 20 mM NAM generation 20, #1","z-score 20 mM NAM generation 5, #2","z-score 20 mM NAM generation 20, #2",generation 5 mean,generation 20 mean,Description
0,YJR030C::chr00_13,YJR030C,NaN,-5.384120,NaN,-7.253220,NaN,-6.318670,Putative protein of unknown function; expressi...
1,YOR291W::chr15_4,YPK9,NaN,-0.898496,NaN,-1.360890,NaN,-1.129693,Vacuolar protein with a possible role in seque...
2,YGL195W::chr7_3,GCN1,NaN,-1.176040,NaN,0.211336,NaN,-0.482352,Positive regulator of the Gcn2p kinase activit...
3,YLR135W::chr12_3,SLX4,25.5204,48.292800,21.8747,51.108200,23.69755,49.700500,Endonuclease involved in processing DNA during...
4,YDR075W::chr4_4,PPH3,16.4712,33.756300,16.4346,34.888900,16.45290,34.322600,Catalytic subunit of protein phosphatase PP4 c...


In [8]:
original_data['orf'] = original_data['strain'].apply(lambda x: x.split(':')[0])

In [9]:
original_data['orf'] = original_data['orf'].astype(str)

In [10]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [11]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [12]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [strain, Gene, z-score 20 mM NAM generation 5, #1, z-score 20 mM NAM generation 20, #1, z-score 20 mM NAM generation 5, #2, z-score 20 mM NAM generation 20, #2, generation 5 mean, generation 20 mean, Description, orf]
Index: []


In [13]:
original_data.set_index('orf', inplace=True)

In [14]:
original_data['data1'] = -original_data[['z-score 20 mM NAM generation 5, #1','z-score 20 mM NAM generation 5, #2']].mean(axis=1)
original_data['data2'] = -original_data[['z-score 20 mM NAM generation 20, #1','z-score 20 mM NAM generation 20, #2']].mean(axis=1)

In [15]:
original_data = original_data[['data1','data2']].copy()

In [16]:
original_data = original_data.groupby(original_data.index).mean()

In [17]:
original_data.shape

(4636, 2)

# Prepare the final dataset

In [18]:
data = original_data.copy()

In [19]:
dataset_ids = [16215, 16214]
datasets = datasets.reindex(index=dataset_ids)

In [20]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [21]:
data.head()

dataset_id,16215,16214
data_type,value,value
orf,,
YAL002W,0.656821,-0.051815
YAL004W,1.143095,-0.731869
YAL005C,-0.435996,1.543101
YAL007C,0.542714,1.780670
YAL008W,-0.185861,0.293978


## Subset to the genes currently in SGD

In [22]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 22


In [23]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

In [24]:
data.head()

,dataset_id,16215,16214
,data_type,value,value
gene_id,orf,,
2,YAL002W,0.656821,-0.051815
1863,YAL004W,1.143095,-0.731869
4,YAL005C,-0.435996,1.543101
5,YAL007C,0.542714,1.780670
6,YAL008W,-0.185861,0.293978


# Normalize

In [25]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [26]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [27]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

In [28]:
data_all.head()


,dataset_id,16215,16214,16215,16214
,data_type,value,value,valuez,valuez
gene_id,orf,,,,
2,YAL002W,0.656821,-0.051815,0.527078,-0.194670
1863,YAL004W,1.143095,-0.731869,0.942936,-0.351404
4,YAL005C,-0.435996,1.543101,-0.407488,0.172918
5,YAL007C,0.542714,1.780670,0.429495,0.227671
6,YAL008W,-0.185861,0.293978,-0.193575,-0.114973


# Print out

In [29]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [30]:
from IO.save_data_to_db3 import *

In [31]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/2 [00:00<?, ?it/s]

Deleting all datasets for PMID 26748095...
Inserting the new data...


100%|██████████| 2/2 [00:14<00:00,  7.44s/it]

Updating the data_modified_on field...
